In [ ]:
import os
import open3d as o3d
from pathlib import Path 
import glob 
from scipy.sparse import csr_matrix 
from scipy.sparse.csgraph import connected_components

from src.similarity_search import SimilaritySearch
from src.extract_individual_teeth import extract_teeth

import glob 

import numpy as np 
import plotly.graph_objects as go

In [ ]:
mesh_folder_paths = glob.glob("/home/shirshak/teeth_similarity_playbook_api/original_data/0AAQ6BO3/*")      #input path of the original raw data, no colored, having labels in json files
outdir = "/home/shirshak/teeth_similarity_playbook_api/individual_teeth/"       #output folder of the indivial teeth

for mesh_folder_path in mesh_folder_paths:
    # print(mesh_folder_path)
    extract_teeth(mesh_folder_path, os.path.join(outdir))

In [ ]:
left_tooth = 14
right_tooth = 16
opposite_tooth = 45

In [ ]:
left_tooth_path, right_tooth_path, opposite_tooth_path = "", "", ""
for file_name in sorted(glob.glob("/home/shirshak/teeth_similarity_playbook_api/individual_teeth/*")):

    label = int(file_name.split("/")[-1].split(".obj")[0].split("fid")[-1])

    if label == left_tooth:
        left_tooth_path = file_name
    
    if  label == right_tooth:
        right_tooth_path = file_name

    if label == opposite_tooth:
        opposite_tooth_path = file_name

In [ ]:
assert left_tooth_path != "" and right_tooth_path != "" and opposite_tooth_path != ""

In [ ]:
teeth_paths = teeth_paths = [left_tooth_path, right_tooth_path, opposite_tooth_path]
tooth_labels = [left_tooth, right_tooth, opposite_tooth]

In [ ]:
similarity_search = SimilaritySearch()
pid, fid_and_similarity_score, dice_dict =  similarity_search.get_similarity_multiple_teeth(teeth_paths, tooth_labels)

In [ ]:
pid

In [ ]:
fid_and_similarity_score

In [ ]:
dice_dict

In [ ]:
fid_and_similarity_score[0][0]

In [ ]:
fid_and_similarity_score[1][0]

In [ ]:
left_tooth_category = "upper" if int(fid_and_similarity_score[0][0].split("fid")[-1]) < 30 else "lower"
right_tooth_category = "upper" if int(fid_and_similarity_score[1][0].split("fid")[-1]) < 30 else "lower"
opposite_tooth_category = "upper" if int(fid_and_similarity_score[2][0].split("fid")[-1]) < 30 else "lower"

In [ ]:
left_tooth_category, right_tooth_category, opposite_tooth_category

In [ ]:
left_similar_teeth = f"/home/shirshak/Teeth3DS_individual_teeth/individual_teeth/{pid}_{left_tooth_category}_{fid_and_similarity_score[0][0]}.obj"
right_similar_teeth = f"/home/shirshak/Teeth3DS_individual_teeth/individual_teeth/{pid}_{right_tooth_category}_{fid_and_similarity_score[1][0]}.obj"
opposite_similar_teeth = f"/home/shirshak/Teeth3DS_individual_teeth/individual_teeth/{pid}_{opposite_tooth_category}_{fid_and_similarity_score[2][0]}.obj"

In [ ]:
similar_teeth_paths = [left_similar_teeth, right_similar_teeth, opposite_similar_teeth]
similar_teeth_paths

In [ ]:
teeth_paths

In [ ]:
def visualize_teeth_comparison(original_path, similar_path, title):
    """
    Visualize two teeth meshes side by side using Plotly with visible coordinate axes.
    
    Args:
        original_path: Path to original tooth mesh
        similar_path: Path to similar tooth mesh
        title: Title for the visualization
    """
    import open3d as o3d
    import numpy as np
    import plotly.graph_objects as go
    
    original_mesh = o3d.io.read_triangle_mesh(original_path)
    similar_mesh = o3d.io.read_triangle_mesh(similar_path)
    original_vertices = np.asarray(original_mesh.vertices)
    original_triangles = np.asarray(original_mesh.triangles)
    
    similar_vertices = np.asarray(similar_mesh.vertices)
    similar_triangles = np.asarray(similar_mesh.triangles)
    
    fig = go.Figure()
    
    # Add original mesh with a blue color
    fig.add_trace(go.Mesh3d(
        x=original_vertices[:, 0],
        y=original_vertices[:, 1],
        z=original_vertices[:, 2],
        i=original_triangles[:, 0],
        j=original_triangles[:, 1],
        k=original_triangles[:, 2],
        color='#1f77b4',  # Blue color
        opacity=0.9,
        name="Original Tooth"
    ))
    
    # Calculate offset for the similar mesh
    offset = [np.max(original_vertices[:, 0]) - np.min(similar_vertices[:, 0]) + 5, 0, 0]
    
    # Add similar mesh with a red color (offset to the right)
    fig.add_trace(go.Mesh3d(
        x=similar_vertices[:, 0] + offset[0],
        y=similar_vertices[:, 1] + offset[1],
        z=similar_vertices[:, 2] + offset[2],
        i=similar_triangles[:, 0],
        j=similar_triangles[:, 1],
        k=similar_triangles[:, 2],
        color='#d62728',  # Red color
        opacity=0.9,
        name="Similar Tooth"
    ))
    
    # Calculate global min and max for axis scaling
    all_vertices = np.vstack([
        original_vertices,
        similar_vertices + np.array(offset)
    ])
    
    x_min, y_min, z_min = np.min(all_vertices, axis=0)
    x_max, y_max, z_max = np.max(all_vertices, axis=0)
    
    # Add some padding
    padding = max(x_max - x_min, y_max - y_min, z_max - z_min) * 0.1
    
    # Update layout with visible axes
    fig.update_layout(
        scene=dict(
            xaxis=dict(
                visible=True,
                showbackground=True,
                backgroundcolor="rgb(230, 230, 230)",
                showgrid=True,
                showticklabels=True,
                title="X Axis",
                range=[x_min - padding, x_max + padding]
            ),
            yaxis=dict(
                visible=True,
                showbackground=True,
                backgroundcolor="rgb(230, 230, 230)",
                showgrid=True,
                showticklabels=True,
                title="Y Axis",
                range=[y_min - padding, y_max + padding]
            ),
            zaxis=dict(
                visible=True,
                showbackground=True,
                backgroundcolor="rgb(230, 230, 230)",
                showgrid=True,
                showticklabels=True,
                title="Z Axis",
                range=[z_min - padding, z_max + padding]
            ),
            aspectmode='data'  # Maintain data aspect ratio
        ),
        title=title,
        scene_camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        ),
        legend=dict(
            x=0.7,
            y=0.9
        )
    )
    
    # Add X, Y, Z axis arrow indicators for better orientation
    axis_length = max(x_max - x_min, y_max - y_min, z_max - z_min) * 0.15
    origin = [x_min - padding/2, y_min - padding/2, z_min - padding/2]
    
    # X-axis arrow (red)
    fig.add_trace(go.Scatter3d(
        x=[origin[0], origin[0] + axis_length],
        y=[origin[1], origin[1]],
        z=[origin[2], origin[2]],
        mode='lines+text',
        line=dict(color='red', width=5),
        text=['', 'X'],
        textposition='middle right',
        showlegend=False
    ))
    
    # Y-axis arrow (green)
    fig.add_trace(go.Scatter3d(
        x=[origin[0], origin[0]],
        y=[origin[1], origin[1] + axis_length],
        z=[origin[2], origin[2]],
        mode='lines+text',
        line=dict(color='green', width=5),
        text=['', 'Y'],
        textposition='middle right',
        showlegend=False
    ))
    
    # Z-axis arrow (blue)
    fig.add_trace(go.Scatter3d(
        x=[origin[0], origin[0]],
        y=[origin[1], origin[1]],
        z=[origin[2], origin[2] + axis_length],
        mode='lines+text',
        line=dict(color='blue', width=5),
        text=['', 'Z'],
        textposition='middle right',
        showlegend=False
    ))
    
    fig.show()

In [ ]:
def visualize_all_teeth_comparisons(teeth_paths, similar_teeth_paths):
    """
    Visualize all pairs of teeth (original and similar).
    
    Args:
        teeth_paths: List of paths to original teeth
        similar_teeth_paths: List of paths to similar teeth
    """
    # Names for visualization based on the file paths
    names = ["Left", "Right", "Opposite"]
    
    # Visualize each pair of teeth
    for i, (original_path, similar_path, name) in enumerate(zip(teeth_paths, similar_teeth_paths, names)):
        title = f"{name} Tooth Comparison"
        visualize_teeth_comparison(original_path, similar_path, title)
        break

In [ ]:
teeth_paths

In [ ]:
similar_teeth_paths

In [ ]:
visualize_all_teeth_comparisons(teeth_paths, similar_teeth_paths)